# BERT Reranking with A2A
This notebook demonstrates a more realistic workflow, where a user downloads all data needed for their experiment and saves it locally once for later use. 

This example is an end-to-end implementation of one of our BERT-based reranking systems submitted to TREC 2021 Clinical Trials track. Specifically we implement the "CSIROmed_abs" model which reranks using "brief_title" and "brief_summary" fields to represent a document. Though we only implement a single BERT reranker, as opposed to the submitted ensemble run, thus the model's performance metrics are slightly different.


## Imports

In [1]:
%pip install --quiet sentencepiece pytorch_lightning transformers

     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 527 kB 74.6 MB/s 
     |████████████████████████████████| 3.5 MB 76.1 MB/s 
     |████████████████████████████████| 952 kB 75.7 MB/s 
     |████████████████████████████████| 134 kB 87.1 MB/s 
     |████████████████████████████████| 397 kB 86.5 MB/s 
     |████████████████████████████████| 596 kB 81.0 MB/s 
     |████████████████████████████████| 829 kB 82.5 MB/s 
     |████████████████████████████████| 1.1 MB 74.1 MB/s 
     |████████████████████████████████| 6.8 MB 71.2 MB/s 
     |████████████████████████████████| 895 kB 78.0 MB/s 
     |████████████████████████████████| 67 kB 7.5 MB/s 
     |████████████████████████████████| 144 kB 86.9 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 271 kB 87.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [8]:
import os
import pickle
import zipfile
import json
from io import BytesIO
import warnings
import requests
import argparse
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
import sentencepiece
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import transformers
transformers.logging.set_verbosity_error() # remove redundant tokenizer warnings

# set seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

## Data cleaning and Model Training Functions

In [10]:
def unzip_and_read_json(response, output_folder="qrel_docs"):
    """Unzip the qrel_docs response and return the json"""
    # extract the zip file contents
    zipped = zipfile.ZipFile(BytesIO(response.content))
    zipped.extractall(output_folder)

    # read the json from zip file
    root, dirs, files = next(os.walk("qrel_docs")) # the unzipped folder contains a single json file, find its name
    file_path = os.path.join(root, files[0])
    with open(file_path, 'r') as file:
        json_dict = json.loads(file.read())
    
    # remove file and folder
    os.remove(file_path)
    os.rmdir(output_folder)
    return json_dict

In [11]:
# This cell contains classes and functions for data cleaning and model training
def prepare_data_and_train_reranker(
        topics, docs, qrels,
        max_epochs=1,
        batch_size=8,
        max_input_size=512,
        model_path="bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12", # Finetune the BlueBERT model from https://arxiv.org/abs/1906.05474
        tokenizer_path="bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12", 
        irrelevance_threshold=0, # labels >= 0 are considered positive examples
        lr=0.00002,
        dropout=0.1,
        weight_decay=0.01,
        warmup_ratio=0.1,
        clip_grad_val=1
    ):
    topics, docs, qrels = transform_training_data(topics, docs, qrels)
    
    # Initialise dataset, dataloader and model
    train_dataset = IRDataset(qrels, docs, topics, tokenizer_path=tokenizer_path, 
        max_input_size=max_input_size, irrelevance_threshold=irrelevance_threshold)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0, pin_memory=True, shuffle=True)
    total_steps = len(train_dataloader)
    model = BERTReranker(model_path=model_path, lr=lr, dropout=dropout, weight_decay=weight_decay, 
                         warmup_ratio=warmup_ratio, total_steps=total_steps)

    # Train with pytorch lightning
    trainer = pl.Trainer(
        logger=True, enable_progress_bar=True, gpus=1, max_epochs=max_epochs, 
        limit_val_batches=0, # we do not use validation, preferring to use the entire sigir_ct dataset for training
        gradient_clip_val=clip_grad_val, gradient_clip_algorithm="value", # clip the value of each parameter's gradient
    )
    trainer.fit(model, train_dataloaders=train_dataloader)
    return model

def get_df(list_of_dicts, relevant_cols):
    """Convert the topic and document dictionaries into a pandas dataframe"""
    rows = [[d[col] for col in relevant_cols if col in d] for d in list_of_dicts] # form 2 dimensional array
    df = pd.DataFrame(rows, columns=relevant_cols)
    df = df.dropna(axis=0) # drop rows that have na values (e.g. in sigir_ct 9 trials lack inclusion criteria)
    return df

def transform_training_data(topics, docs, qrels):
    """Convert topics, documents and qrels into dataframes"""
    topics_df = get_df(topics, ["id", "description"]).rename({"id": "topic_id"}, axis=1)
    topics_df["topic_id"] = topics_df["topic_id"].astype(int)
    docs_df = pd.DataFrame(docs).T
    docs_df = docs_df[["id", "brief_title", "brief_summary"]].rename(columns={"id": "doc_id"})
    docs_df = docs_df.dropna(axis=0) # including any null fields as features ruins the training process, they must be dropped
    qrels_df = pd.DataFrame(qrels, columns=["topic_id", "blank", "doc_id", "relevance_score"]).drop("blank", axis=1)
    return qrels_df, docs_df, topics_df

class IRDataset(torch.utils.data.Dataset):
    def __init__(self, qrels, docs, topics, tokenizer_path="bert-base-uncased", irrelevance_threshold=0, max_input_size=512):
        """
        Load IR task data (from sigir_ct) as dataframes that represent qrels, docs and topics
        Labels is a dummy argument for compatibility with IRDataModule
        Returns tokenized and concatenated topic and document in the form of: "Query: <query> Document: <doc>", also the tokenized mask
        """
        # Create an organised table containing queries and their relevant documents
        self.data = qrels.merge(topics, how="inner", on=["topic_id"])
        self.data = self.data.merge(docs, how="inner", on=["doc_id"])

        # Define tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.max_input_size = max_input_size
        self.irrelevance_threshold = irrelevance_threshold

    def __getitem__(self, index):
        """Tokenize document and query as integer tensors, also return labels"""
        row = self.data.iloc[index]
        label = 0 if row["relevance_score"] <= self.irrelevance_threshold else 1
        query = row["description"]

        # Tokenize data, splitting the topic and document into two sections with different token type IDs https://huggingface.co/docs/transformers/glossary#token-type-ids
        outputs = self.tokenizer(query, f"{row['brief_title']} {row['brief_summary']}", 
            return_tensors='pt', padding='max_length', truncation="longest_first", max_length=self.max_input_size)
        
        return outputs["input_ids"], outputs["attention_mask"], outputs["token_type_ids"], torch.tensor([label])

    def __len__(self):
        return self.data.shape[0]

In [12]:
# Model definition
class BERTReranker(pl.LightningModule):
    """Bert Reranker used in CSIRO's TREC2021 Submission"""
    def __init__(self, model_path="bert-base-uncased", lr=0.00002, dropout=0.1, weight_decay=0.01, total_steps=1000, warmup_ratio=0.1):
        super().__init__()
        self.model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2, 
            hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
        self.lr = lr
        self.dropout = dropout
        self.weight_decay = weight_decay
        self.total_steps = total_steps
        self.warmup_steps = int(warmup_ratio*self.total_steps)

    def configure_optimizers(self):
        # Weight decay of 0.01 is a standard hp https://discuss.huggingface.co/t/does-the-default-weight-decay-of-0-0-in-transformers-adamw-make-sense/1180
        # configure optimizers docs: https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.core.lightning.html#pytorch_lightning.core.lightning.LightningModule.configure_optimizers
        # scheduler doc: https://huggingface.co/docs/transformers/main_classes/optimizer_schedules
        # scheduler usage in pl: https://github.com/PyTorchLightning/pytorch-lightning/issues/328
        self.optimizer = AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.scheduler = get_linear_schedule_with_warmup(self.optimizer, self.warmup_steps, self.total_steps)
        return {
            "optimizer": self.optimizer,
            "lr_scheduler": {
                "scheduler": self.scheduler,
                'interval': 'step',
                'frequency': 1
            }
        }

    def get_inputs(self, batch):
        """
        For each input (except the last label tensor): remove the empty dimension (1) created by the dataloaders batching process
        This creates tensors of size: (batch_size, sequence_length)
        """
        return [tensor.reshape(tensor.shape[0], tensor.shape[-1]) if tensor is not None else None for tensor in batch[:-1]] + [batch[-1]]

    def training_step(self, batch, batch_idx):
        """Train model to generate the label tokens"""
        input_ids, input_mask, token_type_ids, labels = self.get_inputs(batch)
        loss = self.model(input_ids=input_ids, attention_mask=input_mask, token_type_ids=token_type_ids, labels=labels).loss
        return loss

    def forward(self, batch):
        """ Calculate class logits using the [CLS] token's hidden state"""
        # Calculate the first token logits
        with torch.no_grad():
            input_ids, input_mask, token_type_ids, _ = self.get_inputs(batch)
            output = self.model(input_ids=input_ids, attention_mask=input_mask, token_type_ids=token_type_ids)

        # Calculate logits using the linear layer
        logits = output["logits"] #.flip(dims=[1]) # reverse so that the positive class logit is at index 1
        preds_class = torch.argmax(logits, dim=1) # the predicted class: 0 or 1

        return {"preds_class": preds_class, "logits": logits}

In [20]:
# Reranking functions
def rerank_and_save_trec_output(init_retrieval, topics, bm25docs, reranker, rerank_limit=100, model_weight=0.9, max_input_size=512, batch_size=8):
    # Put data in dataframes
    docs_df = pd.DataFrame(bm25docs).T.rename(columns={"id": "doc_id"})
    topics_df = get_df(topics, ["id", "description"]).rename({"id": "topic_id"}, axis=1)
    topics_df["topic_id"] = topics_df["topic_id"].astype(int)
    
    # perform reranking
    rank_dfs = rerank(init_retrieval, topics_df, docs_df, reranker, rerank_limit=rerank_limit, model_weight=model_weight, max_input_size=max_input_size, batch_size=batch_size)
    
    # write results to file
    output_file = "reranker.results"
    dfs_to_trec(rank_dfs, output_file)
    return output_file

# To score documents retrieved by BM25 we need the text of those documents
# First we define a function to convert BM25 results into a dataframe
def api_results_to_dfs(api_dict):
    """convert results from the api "/bm25/{dataset}" call to topic dfs"""
    topic_dfs = []
    for key, value in api_dict["rankings"].items():
        df = pd.DataFrame(value, columns=["doc_id", "score", "ignore", "ignore", "ignore"]).drop(columns=["ignore"])
        df["topic_id"] = int(key)
        df["rank"] = np.arange(df.shape[0]) + 1 # scores are already sorted in descending order, so the rank simply goes from 1 to 1000
        topic_dfs.append(df.copy())
    return topic_dfs

# This function converts the dataframes into the TREC result format
def df_to_trec(df, topic_id=0, file_name=None, rank_col="rank", run_name="ct2021_test"):
    """
    convert a single dataframe to the trec results format for scoring. 
    if file_name is passed, write the results to file, otherwise return the dataframe
    """
    # Convert the ranking to a score, so other ranking columns can be used
    df = df.copy()
    trec_df = df[["doc_id", rank_col]].copy()
    assert not np.any(trec_df[rank_col] == 0), "ranks starting at 0 cause a division by zero error!"
    trec_df["score"] = 1/trec_df[rank_col]
    trec_df = trec_df.sort_values("score", ascending=False)
    trec_df = trec_df[:1000]

    # Add columns in the TREC Format
    trec_df["Q0"] = "Q0"
    trec_df["run_name"] = run_name
    if "topic_id" in df.columns:
        trec_df["topic_id"] = df["topic_id"]
    else:
        trec_df["topic_id"] = topic_id + 1
        warnings.warn(f"No `topic_id` column found in `df`, setting topic_id as {topic_id} which may not be correct")
    trec_df = trec_df.rename(columns={"doc_id": "document_id"})

    trec_df = trec_df.reset_index()
    trec_df = trec_df.reindex(columns=["topic_id", "Q0", "document_id", rank_col, "score", "run_name"]) # reorder the columns
    
    if file_name is not None:
        trec_df.to_csv(file_name, sep="\t", header=False, index=False)
    return trec_df

def dfs_to_trec(dfs, file_name, rank_col="rank", run_name="ct2021_test"):
    """Takes in a list of query dfs and converts them to the TREC results fromat for scoring"""
    results = []
    for topic_id, df in enumerate(dfs):
        results.append(df_to_trec(df, topic_id=topic_id, rank_col=rank_col, run_name=run_name))
    
    results = pd.concat(results, axis=0)
    results.to_csv(file_name, sep="\t", header=False, index=False)

# A new dataset to represent documents without labels is necessary
class RerankDataset(torch.utils.data.Dataset):
    def __init__(self, topic, docs, tokenizer_path="bert-base-uncased", max_input_size=512):
        """
        Dataset for reranking of ct2021 that concatenates the topic and all 1000 documents
        Returns tokenized and concatenated topic and document in the form of: "Query: <query> Document: <doc>", also the tokenized mask
        """
        self.topic = topic
        self.docs = docs
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.max_input_size = max_input_size
        self.dummy = torch.tensor([0])

    def __getitem__(self, index):
        """Tokenize document and query as integer tensors, also return dummy labels"""
        doc = self.docs.iloc[index]
        query = self.topic

        # Tokenize data, splitting the topic and document into two sections with different token type IDs https://huggingface.co/docs/transformers/glossary#token-type-ids
        outputs = self.tokenizer(query, f"{doc['brief_title']} {doc['brief_summary']}",
            return_tensors='pt', padding='max_length', truncation="longest_first", max_length=self.max_input_size)
        
        return outputs["input_ids"], outputs["attention_mask"], outputs["token_type_ids"], self.dummy

    def __len__(self):
        return self.docs.shape[0]

def rerank(init_retrieval, topics, init_docs_df, model, tokenizer_path="bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12", 
           rerank_limit=100, model_weight=0.9, max_input_size=512, batch_size=8):
    """Use `model` to perform reranking of the initially retrieved documents for each topic"""
    trainer = pl.Trainer(gpus=1, enable_progress_bar=True)
    init_dfs = pd.concat(api_results_to_dfs(init_retrieval))
    init_dfs = init_dfs[["topic_id", "doc_id", "score", "rank"]].rename(columns={"score": "bm25_score"}).copy()

    topic_dfs = []
    for i, row in topics.iterrows():
        # Get ranking df for this topic id
        topic_id = int(row["topic_id"])
        init_df = init_dfs[init_dfs["topic_id"] == topic_id].copy()
        
        # Download documents
        ranks_df = pd.merge(init_df, init_docs_df, how="left", on="doc_id") # join Bm25 scores with the docs needed for reranking
        assert init_df.shape[0] == ranks_df.shape[0], "There are duplicate documents in init_docs_df"

        # rerank with model, first create a dataset, then pass all items to the model's forward function
        top_docs_df = ranks_df.iloc[:rerank_limit].copy() # the top documents, in the same order as ranks_df
        topic_dataset = RerankDataset(row["description"], top_docs_df, tokenizer_path=tokenizer_path, max_input_size=max_input_size)        
        topic_dataloader = torch.utils.data.DataLoader(topic_dataset, batch_size=batch_size*2, num_workers=0, pin_memory=True, shuffle=False)
        output_batches = trainer.predict(model, topic_dataloader)
        output_logits = torch.cat([batch["logits"] for batch in output_batches], dim=0)  # output comes in batches, join the logits from each batch together
        pos_class_scores = torch.nn.functional.softmax(output_logits, dim=1)[:, 1]

        # Then normalize and interpolate the scores
        model_scores = pos_class_scores.cpu().detach().numpy().astype(np.float64)
        ranks_df["model_score"] = np.concatenate([model_scores, np.zeros(1000 - model_scores.shape[0])]) # fill up to the 1000th rank with 0 scores so that we can add to df
        ranks_df["bm25_score"] = ranks_df["bm25_score"].astype(np.float64)
        ranks_df["bm25_score"] = ranks_df["bm25_score"]/np.max(ranks_df["bm25_score"])
        ranks_df["score"] = model_weight*ranks_df["model_score"] + (1-model_weight)*ranks_df["bm25_score"]

        # Convert to topic_df format
        ranks_df = ranks_df.sort_values("score", axis=0, ascending=False) # resort by scores
        ranks_df["rank"] = np.arange(ranks_df.shape[0]) + 1 # add the rank column, must start from 1
        topic_dfs.append(ranks_df)
        
        print(f"Topic {i} Rankings:")
        print(ranks_df[["topic_id", "doc_id", "model_score", "bm25_score", "score", "rank"]])
    return topic_dfs

## Reranker training
Get qrel data and document content for docs present in the qrels. Then train a reranker

In [14]:
# Download or load from file the training data: topics, qrels and document content for docs present in the qrels
if not os.path.exists("api_train_data"):
    qrels = requests.get('https://a2a.csiro.au/api/qrels/sigir_ct').json()
    topics = requests.get('https://a2a.csiro.au/api/topics/sigir_ct').json()
    response = requests.get('https://a2a.csiro.au/api/qrel_docs/sigir_ct')
    qrel_docs = unzip_and_read_json(response)

    # Store in pickle format
    with open("api_train_data", "wb") as file:
        pickle.dump((qrels, topics, qrel_docs), file)
else:
    with open("api_train_data", "rb") as file:
        qrels, topics, qrel_docs = pickle.load(file)

In [15]:
reranker = prepare_data_and_train_reranker(topics, qrel_docs, qrels, max_epochs=1)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Missing logger folder: /content/lightning_logs

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 109 M 
--------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.935   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have m

Training: 0it [00:00, ?it/s]

## Reranking Inference

In [16]:
# Download or load from file reranking data: the initial retrieval and text of the documents retrieved
rerank_limit=100 # if changed: the api_inference_data file must be deleted
if not os.path.exists("api_inference_data"):
    bm25_baseline = requests.get('https://a2a.csiro.au/api/bm25/ct2021').json()
    dfs_to_trec(api_results_to_dfs(bm25_baseline), 'bm25.results') # Save bm25 results to disk
    files = {'file': open('bm25.results','rb')}
    bm25_docs = requests.post(f'https://a2a.csiro.au/api/result_docs/ct2021?k={rerank_limit}', files=files).json()
    test_topics = requests.get('https://a2a.csiro.au/api/topics/ct2021').json()

    # Store in pickle format
    with open("api_inference_data", "wb") as file:
        pickle.dump((bm25_baseline, rerank_limit, bm25_docs, test_topics), file)
else:
    with open("api_inference_data", "rb") as file:
        bm25_baseline, rerank_limit, bm25_docs, test_topics = pickle.load(file)

In [27]:
reranked_output_path = rerank_and_save_trec_output(bm25_baseline, test_topics, bm25_docs, reranker, rerank_limit=rerank_limit, model_weight=0.9)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Predicting: 0it [00:00, ?it/s]

Topic 0 Rankings:
     topic_id       doc_id  model_score  bm25_score     score  rank
56          1  NCT00004868     0.554806    0.737139  0.573040     1
0           1  NCT00783393     0.519017    1.000000  0.567115     2
3           1  NCT00004259     0.528630    0.897493  0.565516     3
19          1  NCT00052624     0.534016    0.782074  0.558821     4
44          1  NCT02796261     0.532122    0.748154  0.553725     5
..        ...          ...          ...         ...       ...   ...
995         1  NCT01240590     0.000000    0.479609  0.047961   996
996         1  NCT03901573     0.000000    0.479584  0.047958   997
997         1  NCT03047473     0.000000    0.479568  0.047957   998
998         1  NCT03794024     0.000000    0.479532  0.047953   999
999         1  NCT02202031     0.000000    0.479504  0.047950  1000

[1000 rows x 6 columns]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Predicting: 0it [00:00, ?it/s]

...

Predicting: 0it [00:00, ?it/s]

Topic 73 Rankings:
     topic_id       doc_id  model_score  bm25_score     score  rank
0          74  NCT02802384     0.501104    1.000000  0.550993     1
81         74  NCT00480662     0.524403    0.583299  0.530293     2
12         74  NCT00774020     0.516402    0.653021  0.530064     3
10         74  NCT00051636     0.512520    0.657595  0.527028     4
13         74  NCT00740129     0.437023    0.650924  0.458413     5
..        ...          ...          ...         ...       ...   ...
995        74  NCT03325816     0.000000    0.468113  0.046811   996
996        74  NCT04397445     0.000000    0.468109  0.046811   997
997        74  NCT01518517     0.000000    0.468075  0.046808   998
998        74  NCT01407796     0.000000    0.468065  0.046807   999
999        74  NCT03068351     0.000000    0.468040  0.046804  1000

[1000 rows x 6 columns]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Predicting: 0it [00:00, ?it/s]

Topic 74 Rankings:
     topic_id       doc_id  model_score  bm25_score     score  rank
63         75  NCT02749214     0.588632    0.730803  0.602849     1
68         75  NCT00004387     0.587073    0.726666  0.601032     2
27         75  NCT02384421     0.576689    0.796174  0.598637     3
53         75  NCT03655171     0.582478    0.740304  0.598260     4
3          75  NCT01550484     0.564900    0.884546  0.596864     5
..        ...          ...          ...         ...       ...   ...
995        75  NCT03545477     0.000000    0.489148  0.048915   996
996        75  NCT03000569     0.000000    0.489052  0.048905   997
997        75  NCT01799681     0.000000    0.488716  0.048872   998
998        75  NCT03621046     0.000000    0.488494  0.048849   999
999        75  NCT04505527     0.000000    0.488165  0.048817  1000

[1000 rows x 6 columns]


## Reranking Evaluation

In [28]:
files = {'file': open(reranked_output_path,'rb')}
reranked_results_graded = requests.post('https://a2a.csiro.au/api/eval/ct2021', files=files).json()
files = {'file': open(reranked_output_path,'rb')}
reranked_results_binary = requests.post('https://a2a.csiro.au/api/eval/ct2021_eligible', files=files).json()
print("Reranking Results:", reranked_results_graded['all'], reranked_results_binary['all'])

Reranking Results: {'bpref': 0.3937, 'ndcg_cut_10': 0.4644} {'P_10': 0.272, 'recip_rank': 0.5101}
